In [66]:
import geemap
import ee

In [67]:
Map= geemap.Map(center=[40,-100], zoom=4)

In [68]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [65]:
region_shp= '/Users/allen/OneDrive - University of Oklahoma/CRESTHH/Case study - Harvey/Houston_basin.shp'
region= geemap.shp_to_ee(region_shp)

In [69]:
Map.addLayer(region)

In [71]:
sent1= ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20170829T002620_20170829T002645_018131_01E74D_D734').clip(region)
Map.addLayer(sent1.select('VV'), {'min':-30, 'max':0}, 'SAR')
# Map.setCenter(81.26, 6.28,11)

In [73]:
#mask out water bodies
#import Hansen et al. forest change dataset
hansenImage= ee.Image('UMD/hansen/global_forest_change_2019_v1_6').clip(region)
datamask= hansenImage.select('datamask')
mask= datamask.eq(1)
sent1= sent1.updateMask(mask)
Map.addLayer(sent1, {'min':-30, 'max':0}, 'hansen corrected')

In [74]:
# filter out speckle noise
sent1= sent1.focal_median(30, 'circle', 'meters')

#mask seasonal water
mask02= sent1.select('VV').gt(-16)
sent1= sent1.updateMask(mask02)

Map.addLayer(sent1, {'min':-30, 'max':0}, 'water masked')

In [75]:
# compute the histogram
histogram= sent1.select('VV').reduceRegion(**{'reducer':
                                            (ee.Reducer.histogram(255,2)
                                                 .combine('mean', None, True)
                                                 .combine('variance', None, True)),
                                           'geometry':region,
                                            'scale':30,
                                           'bestEffort':True})

In [76]:
def otsu(histogram):
    counts= ee.Array(ee.Dictionary(histogram).get('histogram'))
    means= ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size= means.length().get([0])
    total= counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum= means.multiply(counts).reduce(ee.Reducer.sum(),[0]).get([0])
    mean= sum.divide(total)
    
    indices= ee.List.sequence(1, size)
    def _lambda(i):
        aCounts= counts.slice(0,0,i)
        aCount= aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans= means.slice(0,0,i)
        aMean= aMeans.multiply(aCounts).reduce(ee.Reducer.sum(),[0]).get([0]).divide(aCount)
        bCount=total.subtract(aCount)
        bMean= sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(bCount.multiply(bMean.subtract(mean).pow(2)))
    
    bss= indices.map(_lambda)
    return means.sort(bss).get([-1])

In [77]:
threshold= otsu(histogram.get('VV_histogram'))

In [78]:
classA = sent1.select('VV').lt(threshold);

Map.addLayer(classA.mask(classA), {'palette': 'blue'}, 'Flood');

In [79]:
#add visual inspection
rgbVis = {
  'min': 0.0,
  'max': 0.3*10000,
  'bands': ['B4', 'B3', 'B2'],
};

In [80]:
dataset = ee.ImageCollection('COPERNICUS/S2').filterBounds(gns).filterDate('2017-08-25', '2017-08-30');
Map.addLayer(dataset.mosaic().clip(region), rgbVis, 'RGB',0);